# Анализ проблемы

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px
import numpy as np


# Затраты на тушение пожаров

Данные приведены до 2012 года. Источник данных ЕМИСС. Данные размещены Рослесхозом

In [3]:
data = pd.read_csv('data/data_cost_2012/Данные-Tаблица 1.csv', sep=';')
data = data.T.reset_index()
data = data.rename(columns={'index': 'год', 0: 'затраты (тыс руб)'})
data = data[1::]
data = data.dropna()
data['затраты (тыс руб)'] = data['затраты (тыс руб)'].apply(lambda x: x.replace(',', '.'))
data['затраты (тыс руб)'] = data['затраты (тыс руб)'].astype(float)
data

,год,затраты (тыс руб)
1,2000,162168.00
2,2001,231376.70
3,2002,610450.20
4,2003,2408986.40
5,2004,836933.40
6,2005,392279.90
7,2006,776808.60
8,2007,568668.70
9,2008,1063683.95
10,2009,843577.40


In [4]:
fig = px.bar(data, x='год', y='затраты (тыс руб)',
             title='Затраты на тушение пожаров до 2012 г')
fig.show()

fig.write_image('img/затраты.jpg')

# Ущерб от пожаров

Приведены данные с 2018 по 2024 гг. Источник данных - ЕМИСС. Данные размещены Рослесхозом

Приведен прямой ущерб, связанный с потерями древесины. Не приведен ущерб имуществу, расходы на лесовосстановление

In [5]:
damage = pd.read_csv('data/damage/Данные-Tаблица 1.csv', sep=';')
damage = damage.T.reset_index()
damage = damage.rename(columns={'index': 'год', 0: 'ущерб (млрд руб)'})

damage = damage[2::]
damage['ущерб (млрд руб)'] = damage['ущерб (млрд руб)'].apply(lambda x: x.replace(',', '.'))
damage['ущерб (млрд руб)'] = damage['ущерб (млрд руб)'].astype(float)


fig = px.bar(damage, x='год', y='ущерб (млрд руб)', title='Ущерб от пожаров, млрд руб')
fig.show()
fig.write_image('img/ущерб.jpg')

# Анализ оперативных данных МЧС по природным пожарам с 2012 по 2021 гг



In [6]:
thermopoints = pd.read_csv('data/thermopoints.csv', sep=';')
thermopoints.head()

,dt,type_name,type_id,lon,lat
0,2012-03-13,Природный пожар,4,131.5866,47.8662
1,2012-03-13,Природный пожар,4,131.5885,47.8809
2,2012-03-13,Лесной пожар,3,131.9871,48.4973
3,2012-03-13,Природный пожар,4,131.9031,43.6277
4,2012-03-13,Природный пожар,4,131.5706,47.8581


In [7]:
thermopoints['dt'] = pd.to_datetime(thermopoints['dt'], yearfirst=True)
thermopoints.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660254 entries, 0 to 660253
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   dt         660254 non-null  datetime64[ns]
 1   type_name  660254 non-null  object        
 2   type_id    660254 non-null  int64         
 3   lon        660254 non-null  float64       
 4   lat        660254 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 25.2+ MB


In [8]:
thermopoints['год'] = thermopoints['dt'].dt.year
thermopoints['месяц'] = thermopoints['dt'].dt.month

In [9]:
thermopoints['type_name'].unique()

array(['Природный пожар', 'Лесной пожар', 'Контролируемый пал',
       'Неконтролируемый пал', 'Торфяной пожар'], dtype=object)

Так как нормально система сбора оперативной информации в МЧС заработала только в 2019 году, то ниже приводится статистика за 2021 год, как наиболее достоверная

In [10]:
wildfire_type_count = thermopoints[thermopoints['год']==2021]['type_name'].value_counts().reset_index()
wildfire_type_count['Доля от общего количества, %'] = wildfire_type_count['count'] / thermopoints[thermopoints['год']==2021].shape[0] * 100
wildfire_type_count = wildfire_type_count.rename(columns={'type_name': 'Тип пожара'})
fig = px.bar(
    wildfire_type_count, x='Тип пожара', y='Доля от общего количества, %', color='Тип пожара',
    title='Распределение пожаров по типам в 2021 г')
fig.show()
fig.write_image('img/типы 2021 г.jpg')

# Распределение пожаров по месяцам

In [11]:
month = {
    1: 'январь',
    2: 'февраль',
    3: 'март',
    4: 'апрель',
    5: 'май',
    6: 'июнь',
    7: 'июль',
    8: 'август',
    9: 'сентябрь',
    10: 'октябрь',
    11: 'ноябрь',
    12: 'декабрь'
}

def transform_month(x):
    return month[x]


wildfire_by_month = thermopoints['месяц'].value_counts().reset_index()
wildfire_by_month = wildfire_by_month.sort_values(by='месяц')
wildfire_by_month['Доля от общего количества, %'] = wildfire_by_month['count'] / thermopoints.shape[0] * 100
wildfire_by_month['месяц'] = wildfire_by_month['месяц'].apply(transform_month)

fig = px.bar(wildfire_by_month, x='месяц', y='Доля от общего количества, %',
             title='Распределение пожаров по месяцам')
fig.show()
fig.write_image('img/распределение месяцы.jpg')

Обнаружено наличие двух пиков по природным пожарам - в апреле и июле

# Анализ типов пожаров в апреле

In [12]:
thermopoints['месяц'] = thermopoints['месяц'].apply(transform_month)

wildfire_type_april = thermopoints[thermopoints['месяц']=='апрель']['type_name'].value_counts().reset_index()
wildfire_type_april['Доля от общего количества, %'] = wildfire_type_april['count'] / thermopoints[thermopoints['месяц']=='апрель'].shape[0] * 100
wildfire_type_april = wildfire_type_april.rename(columns={'type_name': 'Тип пожара'})

In [13]:
fig = px.bar(wildfire_type_april, x='Тип пожара', y='Доля от общего количества, %', 
             title='Распределение пожаров по типам в апреле')
fig.show()
fig.write_image('img/распределение апрель.jpg')

Более 60% пожаров в апреле - антропогенные, связанные с хозяйственной деятельностью человека

# Анализ типов пожаров в июле

In [14]:
wildfire_type_july = thermopoints[thermopoints['месяц']=='июль']['type_name'].value_counts().reset_index()
wildfire_type_july['Доля от общего количества, %'] = wildfire_type_july['count'] / thermopoints[thermopoints['месяц']=='июль'].shape[0] * 100
wildfire_type_july = wildfire_type_july.rename(columns={'type_name': 'Тип пожара'})

In [15]:
fig = px.bar(wildfire_type_july, x='Тип пожара', y='Доля от общего количества, %', 
             title='Распределение пожаров по типам в июле')
fig.show()
fig.write_image('img/распределение июль.jpg')

Более 70% пожаров - лесные. В  первую очередь, связано это с природными факторами - жара, сухие грозы, доля антропогенного фактора неизвестна.

Таким образом, в наличии два пожароопасных периода в течении которых необходимо привлекать большие силы, в том числе и добровольцев, и средства для профилактики и тушения природных пожаров.

# Отображение данных на картах

In [19]:
df = thermopoints[(thermopoints['год']==2021) & (thermopoints['месяц']=='апрель')]

fig = px.scatter_map(df, lat='lat', lon='lon', color='type_name',
                     color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=2)

fig.show()
fig.write_html('maps/map_april_2021.html')

In [20]:
df = thermopoints[(thermopoints['год']==2021) & (thermopoints['месяц']=='июль')]

fig = px.scatter_map(df, lat='lat', lon='lon', color='type_name',
                     color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=2)

fig.show()
fig.write_html('maps/map_july_2021.html')